Test simple convolution

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os, cv2, random
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

TRAIN_DIR = '../input/train/'
TEST_DIR = '../input/test_stg1/'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
ROWS = 720
COLS = 1280
CHANNELS = 3

In [ ]:
def get_images(fish,dirr=TRAIN_DIR):
    """Load files from train folder"""
    fish_dir = dirr +'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im


files = []
y_all = []
for i,fish in enumerate(FISH_CLASSES):
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(i, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

In [ ]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(files): 
    #X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

In [ ]:
plt.imshow(X_all[3])

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
def CNN(inputs, is_training = True,num_classes=8):        
        batch_norm_params = {'is_training': is_training, 'decay': 0.9, 'updates_collections': None}
        with slim.arg_scope([slim.conv2d, slim.fully_connected],
                            normalizer_fn=slim.batch_norm,
                            normalizer_params=batch_norm_params):
            #x = tf.reshape(inputs, [-1, ROWS, COLS, CHANNELS])

            # For slim.conv2d, default argument values are like
            # normalizer_fn = None, normalizer_params = None, <== slim.arg_scope changes these arguments
            # padding='SAME', activation_fn=nn.relu,
            # weights_initializer = initializers.xavier_initializer(),
            # biases_initializer = init_ops.zeros_initializer,
            layers = 8
            net = inputs
            for i in range(layers):
                net = slim.conv2d(net, 8*(2**(i)), [5, 5], scope='conv{}'.format(i))
                net = slim.max_pool2d(net, [2, 2], scope='pool{}'.format(i))
            print(net.get_shape().as_list())
            net = slim.flatten(net, scope='flatten3')

            # For slim.fully_connected, default argument values are like
            # activation_fn = nn.relu,
            # normalizer_fn = None, normalizer_params = None, <== slim.arg_scope changes these arguments
            # weights_initializer = initializers.xavier_initializer(),
            # biases_initializer = init_ops.zeros_initializer,
            net = slim.fully_connected(net, 1024, scope='fc3')
            net = slim.dropout(net, is_training=is_training, scope='dropout3')  # 0.5 by default
            outputs = slim.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None, scope='fco')
        return outputs 

In [ ]:
tf.__version__

In [ ]:
inp = tf.placeholder(tf.float32, [None, ROWS, COLS, CHANNELS])
lab = tf.placeholder(tf.float32, [None, 8])

logits = CNN(inp)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=lab, logits=logits))
arg_predict = tf.argmax(logits,1)
correct_prediction = tf.equal(arg_predict, tf.argmax(lab,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
learning_rate = 0.01
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
saverM = tf.train.Saver(tf.trainable_variables())
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

tloss= 0
tacc = 0
display_step = 2
global_step = 0
epoch = 110
batch_size = 2
def norm(batch):
    m = np.amax(batch)
    mi = np.amin(batch)
    batch = (batch - mi) / (m-mi)
    return  (batch-0.5)
    
def to_one_hot(batch,cl=8):
    y = np.zeros((len(batch),cl))
    #batch = [np.int32(i) for i in batch]
    y[batch] = 1.0
    return y

def  get_batch(batch_size):
    X_all = np.ndarray((batch_size, ROWS, COLS, CHANNELS), dtype=np.uint8)
    ids_train = random.sample(range(0, len(files)), batch_size)
    for i, im in enumerate(files[ids_train]): 
        X_all[i] = read_image(TRAIN_DIR+im)
        
    return X_all, y_all[ids_train]
    
for i in range(global_step,epoch):
            
            #batch_xs = X_all[ids_train]
            #batch_ys = y_all[ids_train]
            batch_xs, batch_ys = get_batch(batch_size)
            _,loss, acc = sess.run([train_step,cross_entropy,accuracy ],
                                        feed_dict={inp: norm(batch_xs), 
                                                   lab: to_one_hot(batch_ys)})
            tloss += loss
            tacc += acc
            if i% display_step == 0 and i>0:
                print('step:',i, ' loss train:',round(tloss/display_step,4), 
                      ', acc train:', round(tacc/display_step,4))
                tloss= 0
                tacc = 0
if False:     
    model_dir = ""
    if not os.path.isdir(model_dir) and False:
                  os.makedirs(model_dir)
    checkpoint_file = os.path.join(model_dir, 'checkpoint')
    saverM.save(sess, checkpoint_file, global_step=epoch)
    print('model saved!')

In [ ]:
batch_size

In [ ]:
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)
id_img = []

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    id_img.append(im)
test.shape , id_img[0]

In [ ]:
submit = open('submit.SVM.csv','w')
submit.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
i=0
while i<test.shape[0]:
    xx = norm(test[i:i+batch_size])
    y_pred = sess.run([logits],feed_dict={inp: xx, lab: np.zeros((len(xx),8))})
    
    for idx in range(len(y_pred)):
        probs=['%s' % p for p in y_pred[idx]]        
        submit.write('%s,%s\n' % (id_img[(i)+idx],','.join(probs)))
        
    i += batch_size
submit.close()

In [ ]:
!ls